In [ ]:
#!/usr/bin/env python3

"""
Raspberry Pi - Otomatik 'Pcap Yükleme' İstemcisi (Cerrahi Engelleme)
Bu script, paketleri toplar, sunucuya yükler ve sunucudan gelen
'blocked_ips' listesine göre SADECE o IP'leri engeller.
"""

import scapy.all as scapy
import requests
import subprocess
import threading
import time
import os

# --- AYARLAR ---
# Dinlenecek arayüz (korunan ağa bağlı USB adaptörünüz, örn: 'eth1')
LISTEN_INTERFACE = "eth1"

# Sunucunuzun (Colab) Ngrok adresi
LLM_API_URL = "https://unrevolted-caleb-semipreserved.ngrok-free.dev/analyze"

# Paketlerin ne sıklıkla toplanıp gönderileceği (saniye)
BATCH_INTERVAL_SECONDS = 60


# --- Global Değişkenler ve Kilitleme ---
# O anki batch'de toplanan paketler (hafızada)
packet_batch = []
# Hafızadaki listeye aynı anda erişimi engellemek için kilit
batch_lock = threading.Lock()
# Zaten engellenen IP'ler (iptables'ı tekrar tekrar yormamak için)
blocked_ips_cache = set()


def check_privileges():
    """Script'in root (sudo) yetkileriyle çalışıp çalışmadığını kontrol eder."""
    if os.geteuid() != 0:
        print("❌ HATA: Root (sudo) yetkileri gereklidir.")
        print("Lütfen 'sudo python3 firewall_client.py' olarak çalıştırın.")
        exit(1)


def block_ip(ip_address, reason):
    """Verilen IP adresini 'iptables' kullanarak engeller."""
    if ip_address in blocked_ips_cache:
        return

    print(f"🚫 [EYLEM] IP Engelleniyor: {ip_address} (Neden: {reason})")
    try:
        command = ["sudo", "iptables", "-I", "FORWARD", "1", "-s", ip_address, "-j", "DROP"]
        subprocess.run(command, check=True, timeout=5)
        blocked_ips_cache.add(ip_address)
        print(f"✅ [Başarılı] IP {ip_address} 'iptables' tarafından engellendi.")
    except Exception as e:
        print(f"❌ [iptables Hatası] IP {ip_address} engellenemedi: {e}")


def batch_sender_loop():
    """
    (Arka Plan Thread'i)
    Her 60 saniyede bir toplanan paketleri .pcap'e yazar,
    sunucuya yükler ve engellenecek IP listesini alır.
    """
    pcap_filepath = "temp_batch.pcap" # Geçici dosya adı

    while True:
        # 'BATCH_INTERVAL_SECONDS' kadar bekle
        print(f"\n... {BATCH_INTERVAL_SECONDS} saniye sonra toplu gönderim yapılacak ...")
        time.sleep(BATCH_INTERVAL_SECONDS)

        batch_to_save = []

        # KİLİTLİ ALAN: Hızlıca veriyi hafızadan kopyala ve listeyi temizle
        with batch_lock:
            if not packet_batch:
                print(f"🕒 [{time.strftime('%H:%M:%S')}] Raporlanacak şüpheli paket yok. Dinlemeye devam.")
                continue

            batch_to_save = packet_batch.copy()
            packet_batch.clear()
        # Kilit serbest

        print(f"📦 [{time.strftime('%H:%M:%S')}] {len(batch_to_save)} paket analiz için hazırlanıyor...")

        # 1. Adım: Hafızadaki paketleri diske .pcap olarak yaz
        try:
            scapy.utils.wrpcap(pcap_filepath, batch_to_save)
        except Exception as e:
            print(f"❌ [Scapy Hatası] Geçici .pcap dosyası yazılamadı: {e}")
            continue

        # 2. Adım: Diskteki .pcap dosyasını sunucuya yükle
        try:
            with open(pcap_filepath, 'rb') as f:
                files = {
                    'pcap_file': (pcap_filepath, f, 'application/vnd.tcpdump.pcap')
                }

                print("... Pcap dosyası Colab'a yükleniyor ve analiz bekleniyor...")
                response = requests.post(LLM_API_URL, files=files, timeout=30)
                response.raise_for_status() # Hata varsa yakala

            # --- 3. Adım: Sunucudan gelen YENİ JSON yanıtını işle ---
            data = response.json()

            # Sunucudan {"blocked_ips": [...], "reason": "..."} formatı bekleniyor
            ips_to_block = data.get('blocked_ips')
            reason = data.get('reason', 'LLM tarafından tanımlandı')

            if ips_to_block is not None: # Eğer 'blocked_ips' anahtarı geldiyse
                if not ips_to_block: # Liste boşsa
                    print(f"🧠 [LLM Kararı] Trafik temiz görünüyor. Engellenecek IP yok.")
                else: # Liste doluysa
                    print(f"🚨 ALARM! Sunucu {len(ips_to_block)} adet IP'nin engellenmesini istedi.")
                    print(f"   Neden: {reason}")

                    # 4. Adım: Sadece listedeki IP'leri engelle
                    for ip in ips_to_block:
                        block_ip(ip, reason)
            else:
                # Sunucu 'blocked_ips' anahtarını göndermediyse bir hata vardır
                print(f"❌ [Sunucu Hatası] Sunucudan anlaşılamayan yanıt: {data.get('error', 'Bilinmeyen hata')}")

        except requests.exceptions.Timeout:
            print("❌ [Colab Hatası] Pcap analiz isteği zaman aşımına uğradı.")
        except requests.exceptions.RequestException as e:
            print(f"❌ [Colab Hatası] Pcap yüklemesi başarısız: {e}")
        except Exception as e:
            print(f"❌ [Bilinmeyen Hata] Analiz yanıtı işlenirken hata: {e}")

        finally:
            # 5. Adım: Geçici dosyayı temizle
            if os.path.exists(pcap_filepath):
                os.remove(pcap_filepath)


def packet_sniffer(packet):
    """
    (Ana Thread - Scapy)
    Bu fonksiyon her paket yakalandığında ÇOK HIZLI çalışır.
    Görevi sadece paketleri hafızadaki global listeye eklemektir.
    """
    # Sadece IP ve TCP katmanları olan paketlerle ilgilen
    if not (packet.haslayer(scapy.IP) and packet.haslayer(scapy.TCP)):
        return
    # Sadece yeni bağlantı girişimlerini (SYN bayrağı) yakala
    if packet[scapy.TCP].flags != 'S':
        return

    try:
        src_ip = packet[scapy.IP].src

        # Lokal, multicast veya loopback IP'leri filtrele
        if packet[scapy.IP].is_private_addr(src_ip) or \
           packet[scapy.IP].is_multicast(src_ip) or \
           packet[scapy.IP].is_loopback(src_ip):
            return

        # Zaten engellenmiş bir IP'den geliyorsa dikkate alma
        if src_ip in blocked_ips_cache:
            return

        # KİLİTLİ ALAN: Paketi hafızadaki listeye ekle
        with batch_lock:
            packet_batch.append(packet)

    except Exception as e:
        print(f"🐞 [Sniffer Hatası] Paket işlenirken hata: {e}")


def main():
    check_privileges()

    print("--- LLM 'Cerrahi Engelleme' İstemcisi Başlatılıyor ---")
    print(f"Dinlenen Arayüz: {LISTEN_INTERFACE}")
    print(f"Colab Sunucu URL: {LLM_API_URL}")
    print(f"Gönderim Aralığı: {BATCH_INTERVAL_SECONDS} saniye")

    print("\n(iptables ve ip_forward ayarlarınızın yapıldığını varsayıyorum...)")

    try:
        # 1. Adım: Arka plan thread'ini (Pcap Gönderici) başlat
        sender_thread = threading.Thread(target=batch_sender_loop, daemon=True)
        sender_thread.start()

        # 2. Adım: Ana thread'de paket dinlemeyi (Sniffer) başlat
        print("Paket dinleyici (Sniffer) ana thread'de başlatıldı...")
        scapy.sniff(
            iface=LISTEN_INTERFACE,
            prn=packet_sniffer,
            filter="tcp and (tcp[tcpflags] & tcp-syn != 0)",
            store=False
        )

    except Exception as e:
        print(f"❌ Arayüz '{LISTEN_INTERFACE}' dinlenirken ciddi bir hata oluştu: {e}")
    finally:
        print("\nFirewall istemcisi durduruluyor...")

if __name__ == "__main__":
    main()